# 5. Topic Modeling with LDA
Use this notebook to practice topic modeling (see lecture slides for reference). First, the usual stuff.

In [1]:
import pandas as pd
reviews_df = pd.read_json('data/json/amazon_reviews.json', lines=True, encoding='utf-8') # to prevent error due to
reviews_df.sample(3)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
63015,en_0199099,product_en_0705066,reviewer_en_0806719,2,"I hate to write bad reviews, but the fact this...","Hidden Thorn, potential dangerous on using the...",en,office_product
132688,en_0789121,product_en_0463343,reviewer_en_0155196,4,You can't see a few of the lighter colored words.,Cute sign,en,home
60394,en_0844842,product_en_0841159,reviewer_en_0242760,2,"This product might be great, had it actually s...",Didn't stick,en,wireless


In [2]:
reviews = reviews_df['review_body'].tolist()
reviews[:3]

["Arrived broken. Manufacturer defect. Two of the legs of the base were not completely formed, so there was no way to insert the casters. I unpackaged the entire chair and hardware before noticing this. So, I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review of part of a chair I never got to sit in. I will go so far as to include a picture of what their injection molding and quality assurance process missed though. I will be hesitant to buy again. It makes me wonder if there aren't missing structures and supports that don't impede the assembly process.",
 'the cabinet dot were all detached from backing... got me',
 "I received my first order of this product and it was broke so I ordered it again. The second one was broke in more places than the first. I can't blame the shipping process as it's shrink wrapped and boxed."]

In [3]:
def remove_puncts(review_text, alphanumeric_only='True'):
    review_text = review_text.replace('-', ' ')
    clean_review_text = ''.join(e for e in review_text if e.isalnum() or e == ' ').lower()
    clean_review_text = ' '.join(clean_review_text.split())
    return clean_review_text

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def get_words_tokenized_nopunct_nostop(reviews, stop_w=stop_words):
    review_words_list = []
    for review in reviews:
        clean_review = remove_puncts(review)
        words = word_tokenize(clean_review.lower())
        words_nostop = [word for word in words if not word in stop_w]
        review_words_list.append(words_nostop)
    return review_words_list

In [5]:
tokenized_reviews_list = get_words_tokenized_nopunct_nostop(reviews)

## Train an LDA Model
LDA models require an estimated number of topics. We can choose this to be 31, equal to the number of product types.

In [6]:
import gensim

dictionary = gensim.corpora.Dictionary(tokenized_reviews_list)
corpus = [dictionary.doc2bow(review) for review in tokenized_reviews_list]
topic_count = 31
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topic_count, id2word=dictionary, passes=50)

## Visualize the LDA Model
We use the famouse [pyLDAvis](https://pypi.org/project/pyLDAvis/) library to make sense of the topics.

First, we set up the notebook to suppress warnings (you'll get a lot due to package deprecations).

In [7]:
import warnings
warnings.filterwarnings('ignore')

## IMPORTANT: Set the slider below to a $\lambda$ value of 0.4
A $\lambda$ of 1 shows all frequently-occurring words, which gives you a global picture of the text in general, but does not help you understand the topics. A $\lambda$ of 0.1 shows only the words unique to each topic, which helps you understand the difference between the topics, but not what the topics are about. A $\lambda$ of 0.4 achieves a good balance. See [this paper](https://aclanthology.org/W14-3110.pdf) for more details.

In [12]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
gensimvis.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11     0.271413 -0.138790       1        1  9.029596
6      0.200890 -0.073008       2        1  7.630016
24     0.149112  0.125008       3        1  5.552652
4      0.108131 -0.268287       4        1  4.715225
30     0.248516  0.110602       5        1  4.385617
2      0.139422  0.165169       6        1  3.977215
29    -0.040361 -0.169284       7        1  3.867335
0      0.093130  0.075038       8        1  3.458748
10     0.123276 -0.058780       9        1  3.406501
25     0.006351  0.102135      10        1  3.233609
17    -0.033782 -0.163315      11        1  3.194356
21    -0.162605  0.008530      12        1  3.059281
12    -0.002814 -0.147279      13        1  2.868731
27     0.016741  0.177118      14        1  2.866547
9     -0.005658 -0.006665      15        1  2.772157
16    -0.008221 -0.010906      16        1  2.763714
14    -0.137599 -0.020138      17        1  2.759279
18    -0.024159 -0.139260      18        1  2.727716
1      0.026999  0.108051      19        1  2.607633
19    -0.075233 -0.123134      20        1  2.542358
5      0.051111  0.143188      21        1  2.486129
8     -0.070407 -0.071337      22        1  2.392495
28    -0.173949  0.061451      23        1  2.302506
26    -0.080629  0.048225      24        1  2.103818
23    -0.040715  0.016755      25        1  2.091454
20    -0.090675  0.108047      26        1  2.046534
15    -0.048852  0.078854      27        1  1.942460
7     -0.110320 -0.041103      28        1  1.933414
13    -0.092305  0.008760      29        1  1.866067
3     -0.157582  0.008522      30        1  1.860702
22    -0.079228  0.085832      31        1  1.556134, topic_info=         Term          Freq         Total Category  logprob  loglift
162     great  48529.000000  48529.000000  Default  30.0000  30.0000
353      good  34518.000000  34518.000000  Default  29.0000  29.0000
1133     love  30188.000000  30188.000000  Default  28.0000  28.0000
39    quality  19447.000000  19447.000000  Default  27.0000  27.0000
416     works  19112.000000  19112.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1272    thats    641.987687   2632.030901  Topic31  -4.4250   2.7520
166    really    868.918604  21416.072288  Topic31  -4.1223   0.9583
767      kind    548.630018   1768.600640  Topic31  -4.5821   2.9924
361    little    718.259248  14974.271507  Topic31  -4.3127   1.1257
1904     cool    550.454365   2704.719193  Topic31  -4.5788   2.5710

[1407 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0        11  0.999625        1
549      11  0.999394       10
2408      8  0.178764      100
2408     12  0.185344      100
2408     24  0.634996      100
...     ...       ...      ...
2205      9  0.998783    youll
1197      2  0.999486    youre
1236     18  0.998005  youtube
2443      2  0.997979     zero
1863     26  0.998378   zipper

[1965 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 7, 25, 5, 31, 3, 30, 1, 11, 26, 18, 22, 13, 28, 10, 17, 15, 19, 2, 20, 6, 9, 29, 27, 24, 21, 16, 8, 14, 4, 23])

## View topic distribution for a document (review)

In [13]:
ldamodel.get_document_topics(corpus[0])

[(1, 0.03990258),
 (5, 0.020167688),
 (9, 0.046473116),
 (11, 0.25684324),
 (13, 0.08034494),
 (14, 0.045582913),
 (15, 0.020696638),
 (17, 0.02063708),
 (18, 0.07433518),
 (19, 0.0727096),
 (23, 0.2571909),
 (29, 0.02953556),
 (30, 0.024823532)]

# Exercise: What are some topics relevant to you as a designer?
Explore the topics and their keywords. 